In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import ast
from datetime import datetime,timedelta,date
import glob
import json
import uuid
import time

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 200)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
tqdm.pandas()

from h3 import h3

In [2]:
from pyhive import presto
presto_connection = presto.connect(
            host='presto-gateway.serving.data.production.internal',
            port=80,
            protocol='http',
            catalog='hive',
            username='manoj.ravirajan@rapido.bike')


In [3]:
from ast import literal_eval


In [4]:
affluence_cb_hex_8_data = pd.read_csv('/Users/rapido/local-datasets/affluence/experiment/exp_affluence_bangalore_link_circuit_break_hex_8_list_v1.csv')

In [5]:
affluence_cb_hex_8_data_summary = affluence_cb_hex_8_data.groupby(['affluence_tag', 'group_tc']).agg({'pickup_hex_8' : 'count', \
                                                          'fe_count' : 'sum', \
                                                          'gross_orders' : 'sum', \
                                                          'net_orders' : 'sum', \
                                                          'surged_fe' : 'sum', \
                                                          'fe_customer_count' : 'sum'
                                                         })
affluence_cb_hex_8_data_summary['fe2rr'] = affluence_cb_hex_8_data_summary['gross_orders']*100.0/affluence_cb_hex_8_data_summary['fe_count']
affluence_cb_hex_8_data_summary['g2n'] = affluence_cb_hex_8_data_summary['net_orders']*100.0/affluence_cb_hex_8_data_summary['gross_orders']
affluence_cb_hex_8_data_summary['fe2net'] = affluence_cb_hex_8_data_summary['net_orders']*100.0/affluence_cb_hex_8_data_summary['fe_count']
affluence_cb_hex_8_data_summary

pickup_hex_8  fe_count  gross_orders  net_orders  \
affluence_tag  group_tc                                                     
High Affluence control             12    167314       29967.0       12005   
               test                13    159772       29755.0       11272   
Low Affluence  control             22    188181       32635.0       12546   
               test                23    193892       31985.0       12466   

                         surged_fe  fe_customer_count      fe2rr        g2n  \
affluence_tag  group_tc                                                       
High Affluence control       73982              49060  17.910635  40.060733   
               test          68565              40879  18.623413  37.882709   
Low Affluence  control       75303              58673  17.342346  38.443389   
               test          81135              64661  16.496297  38.974519   

                           fe2net  
affluence_tag  group_tc            
High Affluence control   7.175132  
               test      7.055053  
Low Affluence  control   6.666986  
               test      6.429352

In [6]:
affluence_cb_hex_8_data.columns

Index(['pickup_cluster', 'pickup_hex_8', 'fe_count', 'gross_orders',
       'net_orders', 'surged_fe', 'surge_percentage', 'discount_amount_fe',
       'discount_gross', 'sub_total_fe', 'fe_customer_count',
       'mean_geo_distance', 'max_geo_distance', 'fe2rr', 'fe2net', 'g2n',
       'income_signal', 'group_tc', 'affluence_tag'],
      dtype='object')

In [7]:
affluence_cb_hex_8_data.pickup_hex_8.nunique()

70

In [8]:
control_affluence_cb_hex_8_data = affluence_cb_hex_8_data[affluence_cb_hex_8_data['affluence_tag'] == 'Low Affluence']
control_affluence_cb_hex_8_data.pickup_hex_8.nunique()

45

In [9]:
control_affluence_cb_hex_8_data.pickup_hex_8.unique()

array(['886014594bfffff', '8860145863fffff', '8861892ca3fffff',
       '8860145829fffff', '8860145a05fffff', '8860169669fffff',
       '8860145943fffff', '8861892dd5fffff', '886014584dfffff',
       '886189244dfffff', '88618925a9fffff', '8860145913fffff',
       '8861892c0bfffff', '8861892cc1fffff', '8860145ad1fffff',
       '88618925e3fffff', '886016966bfffff', '88618920e9fffff',
       '8861892645fffff', '8861892521fffff', '8861892ccbfffff',
       '8861892e63fffff', '8861892f13fffff', '8860145ad9fffff',
       '88618921e3fffff', '8860145b51fffff', '8861892e2bfffff',
       '8861892c8bfffff', '8860145b67fffff', '8861892e2dfffff',
       '8861892db5fffff', '8860145a21fffff', '8861892db7fffff',
       '8860145a29fffff', '8861892c95fffff', '8861892eb1fffff',
       '8861892eb7fffff', '8861892eb5fffff', '8860145b53fffff',
       '8861892e93fffff', '8860145b55fffff', '886014595dfffff',
       '8861892e85fffff', '8860145b59fffff', '8860169663fffff'],
      dtype=object)

In [10]:
affluence_cb_hex_8_data = affluence_cb_hex_8_data[affluence_cb_hex_8_data['group_tc'] == 'test']

In [11]:
affluence_cb_hex_8_data.pickup_hex_8.nunique()

36

In [12]:
affluence_cb_hex_8_data.pickup_hex_8.unique()

array(['88618921d3fffff', '8861892665fffff', '8861892c97fffff',
       '88618920b3fffff', '8861892ed9fffff', '8861892c11fffff',
       '8861892c55fffff', '886189246dfffff', '8861892ee5fffff',
       '88618925bbfffff', '8861892431fffff', '8860145b43fffff',
       '88618925a5fffff', '886014594bfffff', '8861892ca3fffff',
       '8860145a05fffff', '8860145943fffff', '886014584dfffff',
       '88618925a9fffff', '8861892c0bfffff', '8860145ad1fffff',
       '886016966bfffff', '8861892645fffff', '8861892ccbfffff',
       '8861892f13fffff', '88618921e3fffff', '8861892e2bfffff',
       '8860145b67fffff', '8861892db5fffff', '8861892db7fffff',
       '8861892c95fffff', '8861892eb7fffff', '8860145b53fffff',
       '8860145b55fffff', '8861892e85fffff', '8860169663fffff'],
      dtype=object)

In [13]:
affluence_cb_hex_8_data['city'] = 'Bangalore'
affluence_cb_hex_8_data['start_time'] = '0'
affluence_cb_hex_8_data['end_time'] = '2359'
affluence_cb_hex_8_data['pickup_hex8'] = affluence_cb_hex_8_data['pickup_hex_8'] 
affluence_cb_hex_8_data['drop_cluster'] = 'ALL'
affluence_cb_hex_8_data['drop_hex8'] = 'ALL'
affluence_cb_hex_8_data = affluence_cb_hex_8_data[['city','pickup_cluster','drop_cluster','start_time','end_time','pickup_hex8','drop_hex8']]
affluence_cb_hex_8_data['k'] = 1
affluence_cb_hex_8_data['l'] = 1
affluence_cb_hex_8_data

,city,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,k,l
0,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,1,1
2,Bangalore,Electronic City Phase 2,ALL,0,2359,8861892665fffff,ALL,1,1
4,Bangalore,Mathikere,ALL,0,2359,8861892c97fffff,ALL,1,1
6,Bangalore,Marathahalli Village,ALL,0,2359,88618920b3fffff,ALL,1,1
8,Bangalore,Domlur,ALL,0,2359,8861892ed9fffff,ALL,1,1
10,Bangalore,Horamavu,ALL,0,2359,8861892c11fffff,ALL,1,1
12,Bangalore,Horamavu,ALL,0,2359,8861892c55fffff,ALL,1,1
14,Bangalore,Harlur,ALL,0,2359,886189246dfffff,ALL,1,1
16,Bangalore,Banaswadi,ALL,0,2359,8861892ee5fffff,ALL,1,1
18,Bangalore,Metro - Lalbagh,ALL,0,2359,88618925bbfffff,ALL,1,1


In [14]:
affluence_cb_hex_8_data.shape

(36, 9)

In [15]:
df_eligible_strategy = pd.DataFrame({'l': [1,1,1], 'eligible_strategy': ['hot_cold_surge', \
                                                                                 'surge_dashboard_rule', \
                                                                                 'mismatch']})

df_eligible_strategy

,l,eligible_strategy
0,1,hot_cold_surge
1,1,surge_dashboard_rule
2,1,mismatch


In [16]:
df_week = pd.DataFrame({'k': [1,1,1,1,1,1,1], 'week_period': ['Monday','Tuesday',
                                                              'Wednesday','Thursday',
                                                              'Friday','Saturday','Sunday']})

df_week

,k,week_period
0,1,Monday
1,1,Tuesday
2,1,Wednesday
3,1,Thursday
4,1,Friday
5,1,Saturday
6,1,Sunday


In [17]:
affluence_cb_hex_8_data = affluence_cb_hex_8_data.merge(df_eligible_strategy,on='l').drop(columns = 'l',axis =1)
affluence_cb_hex_8_data.shape

(108, 9)

In [18]:
36*3

108

In [19]:
affluence_cb_hex_8_data = affluence_cb_hex_8_data.merge(df_week,on='k').drop(columns = 'k',axis =1)
affluence_cb_hex_8_data.shape

(756, 9)

In [20]:
108*7

756

In [21]:
affluence_cb_hex_8_data

,city,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period
0,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Monday
1,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Tuesday
2,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Wednesday
3,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Thursday
4,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Friday
...,...,...,...,...,...,...,...,...,...
751,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Wednesday
752,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Thursday
753,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Friday
754,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Saturday


In [22]:
city_name = affluence_cb_hex_8_data['city'].unique()[0]

In [23]:
'''
query = """

        SELECT 
            service_detail_id
        FROM 
            datasets.service_mapping
        WHERE 
            city_display_name = '{city}' 
            AND service_level = 'Link'

        """.format(city = city_name)

service_mapping = pd.read_sql(query, presto_connection).iloc[0]
city_service_detail_id = service_mapping[0]
'''

'\nquery = """\n\n        SELECT \n            service_detail_id\n        FROM \n            datasets.service_mapping\n        WHERE \n            city_display_name = \'{city}\' \n            AND service_level = \'Link\'\n\n        """.format(city = city_name)\n\nservice_mapping = pd.read_sql(query, presto_connection).iloc[0]\ncity_service_detail_id = service_mapping[0]\n'

In [24]:
city_service_detail_id = '57370b61a6855d70057417d1'

In [25]:
# Bangalore | Link | service_detail_id = 57370b61a6855d70057417d1

In [26]:
CITY = city_name                                    # FORMAT: 'CITY_NAME', As per record
START_DATE = '20230821'                             # FORMAT: 'YYYYMMDD'
END_DATE = '20230910'                               # FORMAT: 'YYYYMMDD'
SERVICE_DETAIL_ID = city_service_detail_id          # FORMAT: 'SERVICE_DETAIL_ID', As per record
CAPTAIN_PAYOUT = 0                                  # FORMAT: In Rupees

In [27]:
sample = {
    "service_detail_id": "57370b61a6855d70057417d1",
    "end_date": "20220401",
    "city": "Bangalore",
    "run_date": "20220331",
    "updated_at": 1645675999,
    "recommendation_date": "20220401",
    "pickup_cluster": "Paltan Bazaar",
    "drop_cluster": "Paltan Bazaar",
    "end_time": "1030",
    "type": "order_acceptance",
    "captain_payout": 23,
    "start_date": "20220401",
    "created_at": 1645675999,
    "period": "",
    "updated_by": "manoj.ravirajan@rapido.bike",
    "start_time": "1000",
    "pickup_hex8":"88619aa4d9fffff",
    "drop_hex8":"88619aa4d9fffff",
    "start_distance":0,
    "end_distance":100,
    "rule_id": "d2e843de-a3bd-4de9-a2e5-22e41953d8d0",
    "eligible_strategy":'surge_dashboard_rule'
}

In [28]:
config_format = json.loads(json.dumps(sample))
config_format = pd.DataFrame(config_format, index=[0])

In [29]:
configuration = affluence_cb_hex_8_data.copy()

In [30]:
configuration['service_detail_id'] = SERVICE_DETAIL_ID
configuration['start_date'] = START_DATE
configuration['end_date'] = END_DATE
configuration['run_date'] = date.today().strftime('%Y%m%d')
configuration['start_time'] = configuration['start_time']
configuration['created_at'] = int(time.time())
configuration['updated_at'] = int(time.time())
configuration['period'] = ''
configuration['captain_payout'] = CAPTAIN_PAYOUT
configuration['type'] = 'order_acceptance'
configuration['updated_by'] = 'manoj.ravirajan@rapido.bike'
configuration['city'] = CITY
configuration['start_distance'] = 0
configuration['end_distance'] = 100

In [31]:
configuration.start_date.unique()

array(['20230821'], dtype=object)

In [32]:
configuration.end_date.unique()

array(['20230910'], dtype=object)

In [33]:
def rule(configuration):
    rule_id = str(uuid.uuid4())
    return rule_id

configuration['rule_id'] = configuration.apply(rule,axis=1)

In [34]:
def recommend_date(configuration):
    
    firstDay = datetime.strptime(configuration['start_date'], '%Y%m%d') 
    lastDay = datetime.strptime(configuration['end_date'], '%Y%m%d')
    weekDay = configuration['week_period']
    dates = [firstDay + timedelta(days=x) for x in range((lastDay-firstDay).days + 1) if (firstDay + timedelta(days=x)).weekday() == time.strptime(weekDay, '%A').tm_wday]
    r_date = dates[0].strftime("%Y%m%d")
    return r_date

configuration['recommendation_date'] = configuration.apply(recommend_date,axis=1)
configuration['start_date'] = configuration['recommendation_date'].copy()
configuration['end_date'] = configuration['recommendation_date'].copy()

In [35]:
configuration.recommendation_date.unique()

array(['20230821', '20230822', '20230823', '20230824', '20230825',
       '20230826', '20230827'], dtype=object)

In [36]:
configuration_week1 = configuration.copy()

In [37]:
configuration['start_date'] = START_DATE
configuration['end_date'] = END_DATE

In [38]:
def recommend_date(configuration):
    
    firstDay = datetime.strptime(configuration['start_date'], '%Y%m%d') 
    lastDay = datetime.strptime(configuration['end_date'], '%Y%m%d')
    weekDay = configuration['week_period']
    dates = [firstDay + timedelta(days=x) for x in range((lastDay-firstDay).days + 1) if (firstDay + timedelta(days=x)).weekday() == time.strptime(weekDay, '%A').tm_wday]
    r_date = dates[1].strftime("%Y%m%d")
    return r_date

configuration['recommendation_date'] = configuration.apply(recommend_date,axis=1)
configuration['start_date'] = configuration['recommendation_date'].copy()
configuration['end_date'] = configuration['recommendation_date'].copy()

In [39]:
configuration.recommendation_date.unique()

array(['20230828', '20230829', '20230830', '20230831', '20230901',
       '20230902', '20230903'], dtype=object)

In [40]:
configuration_week2 = configuration.copy()

In [41]:
configuration['start_date'] = START_DATE
configuration['end_date'] = END_DATE

In [42]:
configuration.start_date.unique()

array(['20230821'], dtype=object)

In [43]:
configuration.end_date.unique()

array(['20230910'], dtype=object)

In [44]:
def recommend_date(configuration):
    
    firstDay = datetime.strptime(configuration['start_date'], '%Y%m%d') 
    lastDay = datetime.strptime(configuration['end_date'], '%Y%m%d')
    weekDay = configuration['week_period']
    dates = [firstDay + timedelta(days=x) for x in range((lastDay-firstDay).days + 1) if (firstDay + timedelta(days=x)).weekday() == time.strptime(weekDay, '%A').tm_wday]
    r_date = dates[2].strftime("%Y%m%d")
    return r_date

configuration['recommendation_date'] = configuration.apply(recommend_date,axis=1)
configuration['start_date'] = configuration['recommendation_date'].copy()
configuration['end_date'] = configuration['recommendation_date'].copy()

In [45]:
configuration.recommendation_date.unique()

array(['20230904', '20230905', '20230906', '20230907', '20230908',
       '20230909', '20230910'], dtype=object)

In [46]:
configuration_week3 = configuration.copy()

In [47]:
configuration_merge = [configuration_week1, configuration_week2, configuration_week3]
configuration = pd.concat(configuration_merge)

In [48]:
configuration.shape

(2268, 23)

In [49]:
756*3

2268

In [50]:
configuration.rule_id.nunique()

756

In [51]:
def rule(configuration):
    rule_id = str(uuid.uuid4())
    return rule_id

configuration['rule_id'] = configuration.apply(rule,axis=1)

In [52]:
configuration.rule_id.nunique()

2268

In [53]:
configuration.head()

,city,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period,service_detail_id,start_date,end_date,run_date,created_at,updated_at,period,captain_payout,type,updated_by,start_distance,end_distance,rule_id,recommendation_date
0,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Monday,57370b61a6855d70057417d1,20230821,20230821,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,c61b4f35-e22f-4488-8796-f3f5df959c51,20230821
1,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Tuesday,57370b61a6855d70057417d1,20230822,20230822,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,aa5bfc30-210a-4033-a4fc-39b5ab962d8f,20230822
2,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Wednesday,57370b61a6855d70057417d1,20230823,20230823,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4d0077c1-2940-47a3-b478-a25eb4ebab91,20230823
3,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Thursday,57370b61a6855d70057417d1,20230824,20230824,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,9c603e05-a94a-420f-9bdc-9d1e120f6d0f,20230824
4,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Friday,57370b61a6855d70057417d1,20230825,20230825,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,ce0495b3-4dda-42d7-8e99-aab4bb108488,20230825


In [54]:
configuration.shape

(2268, 23)

In [55]:
configuration['recommendation_date'].unique()

array(['20230821', '20230822', '20230823', '20230824', '20230825',
       '20230826', '20230827', '20230828', '20230829', '20230830',
       '20230831', '20230901', '20230902', '20230903', '20230904',
       '20230905', '20230906', '20230907', '20230908', '20230909',
       '20230910'], dtype=object)

In [56]:
week1 = configuration.copy()

In [57]:
week_all = pd.concat([week1]).reset_index(drop=True)

In [58]:
week_all

,city,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period,service_detail_id,start_date,end_date,run_date,created_at,updated_at,period,captain_payout,type,updated_by,start_distance,end_distance,rule_id,recommendation_date
0,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Monday,57370b61a6855d70057417d1,20230821,20230821,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,c61b4f35-e22f-4488-8796-f3f5df959c51,20230821
1,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Tuesday,57370b61a6855d70057417d1,20230822,20230822,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,aa5bfc30-210a-4033-a4fc-39b5ab962d8f,20230822
2,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Wednesday,57370b61a6855d70057417d1,20230823,20230823,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4d0077c1-2940-47a3-b478-a25eb4ebab91,20230823
3,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Thursday,57370b61a6855d70057417d1,20230824,20230824,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,9c603e05-a94a-420f-9bdc-9d1e120f6d0f,20230824
4,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Friday,57370b61a6855d70057417d1,20230825,20230825,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,ce0495b3-4dda-42d7-8e99-aab4bb108488,20230825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Wednesday,57370b61a6855d70057417d1,20230906,20230906,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,932f73d0-8767-45e1-b413-dd918a6931c3,20230906
2264,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Thursday,57370b61a6855d70057417d1,20230907,20230907,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4da2bf9d-6af9-454b-b83e-e9442fbd7af4,20230907
2265,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Friday,57370b61a6855d70057417d1,20230908,20230908,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,d2dc85f0-1e01-46ea-9585-808d8ad90741,20230908
2266,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Saturday,57370b61a6855d70057417d1,20230909,20230909,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,bd5c50dd-0d2c-4e33-9a51-0cc9167b165e,20230909


In [59]:
week_all['city'].unique()

array(['Bangalore'], dtype=object)

In [60]:
week_all['recommendation_date'].unique()

array(['20230821', '20230822', '20230823', '20230824', '20230825',
       '20230826', '20230827', '20230828', '20230829', '20230830',
       '20230831', '20230901', '20230902', '20230903', '20230904',
       '20230905', '20230906', '20230907', '20230908', '20230909',
       '20230910'], dtype=object)

In [61]:
batch = week_all.copy()

In [62]:
all_city_df2 = pd.concat([batch]).reset_index(drop=True)

In [63]:
all_city_df2

,city,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period,service_detail_id,start_date,end_date,run_date,created_at,updated_at,period,captain_payout,type,updated_by,start_distance,end_distance,rule_id,recommendation_date
0,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Monday,57370b61a6855d70057417d1,20230821,20230821,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,c61b4f35-e22f-4488-8796-f3f5df959c51,20230821
1,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Tuesday,57370b61a6855d70057417d1,20230822,20230822,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,aa5bfc30-210a-4033-a4fc-39b5ab962d8f,20230822
2,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Wednesday,57370b61a6855d70057417d1,20230823,20230823,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4d0077c1-2940-47a3-b478-a25eb4ebab91,20230823
3,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Thursday,57370b61a6855d70057417d1,20230824,20230824,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,9c603e05-a94a-420f-9bdc-9d1e120f6d0f,20230824
4,Bangalore,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Friday,57370b61a6855d70057417d1,20230825,20230825,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,ce0495b3-4dda-42d7-8e99-aab4bb108488,20230825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Wednesday,57370b61a6855d70057417d1,20230906,20230906,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,932f73d0-8767-45e1-b413-dd918a6931c3,20230906
2264,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Thursday,57370b61a6855d70057417d1,20230907,20230907,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4da2bf9d-6af9-454b-b83e-e9442fbd7af4,20230907
2265,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Friday,57370b61a6855d70057417d1,20230908,20230908,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,d2dc85f0-1e01-46ea-9585-808d8ad90741,20230908
2266,Bangalore,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,mismatch,Saturday,57370b61a6855d70057417d1,20230909,20230909,20230904,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,bd5c50dd-0d2c-4e33-9a51-0cc9167b165e,20230909


In [64]:
all_city_df2.shape

(2268, 23)

# Saving the file in parquet format

In [65]:
import os
os.getcwd()

'/Users/rapido/code-repository/pricing/affluence/experiment'

In [66]:
configurations_directory = '/Users/rapido/local-datasets/affluence/experiment/experiment_documents/cb_order_acceptance'

!rm -R {configurations_directory}
!mkdir {configurations_directory}

all_city_df2.to_parquet(configurations_directory, partition_cols=['run_date', 'city', 'recommendation_date'])

rm: /Users/rapido/local-datasets/affluence/experiment/experiment_documents/cb_order_acceptance: No such file or directory
mkdir: /Users/rapido/local-datasets/affluence/experiment/experiment_documents: No such file or directory


# Doing sanity checks

In [67]:
df = pd.read_parquet('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/cb_order_acceptance')
df

,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period,service_detail_id,start_date,end_date,created_at,updated_at,period,captain_payout,type,updated_by,start_distance,end_distance,rule_id,run_date,city,recommendation_date
0,Whitefield,ALL,0,2359,88618921d3fffff,ALL,hot_cold_surge,Monday,57370b61a6855d70057417d1,20230821,20230821,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,c61b4f35-e22f-4488-8796-f3f5df959c51,20230904,Bangalore,20230821
1,Whitefield,ALL,0,2359,88618921d3fffff,ALL,surge_dashboard_rule,Monday,57370b61a6855d70057417d1,20230821,20230821,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,bc61aa0a-413c-4d72-ad5f-eed61f87865d,20230904,Bangalore,20230821
2,Whitefield,ALL,0,2359,88618921d3fffff,ALL,mismatch,Monday,57370b61a6855d70057417d1,20230821,20230821,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,ec4d8616-dc5c-47f1-8dc5-5519628c6c01,20230904,Bangalore,20230821
3,Electronic City Phase 2,ALL,0,2359,8861892665fffff,ALL,hot_cold_surge,Monday,57370b61a6855d70057417d1,20230821,20230821,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,5537a69c-fd2c-486a-a8da-6848a63a7eb5,20230904,Bangalore,20230821
4,Electronic City Phase 2,ALL,0,2359,8861892665fffff,ALL,surge_dashboard_rule,Monday,57370b61a6855d70057417d1,20230821,20230821,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,c9345e1f-8aef-43ce-a7f5-ecef64a88d45,20230904,Bangalore,20230821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,Banaswadi,ALL,0,2359,8861892e85fffff,ALL,surge_dashboard_rule,Sunday,57370b61a6855d70057417d1,20230910,20230910,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,d0eb8729-9391-4857-a93e-d8890b58f0b1,20230904,Bangalore,20230910
2264,Banaswadi,ALL,0,2359,8861892e85fffff,ALL,mismatch,Sunday,57370b61a6855d70057417d1,20230910,20230910,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,2d727292-f466-4746-b1a3-ed45e0299782,20230904,Bangalore,20230910
2265,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,hot_cold_surge,Sunday,57370b61a6855d70057417d1,20230910,20230910,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,6fdc8d9a-2c66-440d-b6f9-75d639d6c8d0,20230904,Bangalore,20230910
2266,Yelahanka Traffic Police Station,ALL,0,2359,8860169663fffff,ALL,surge_dashboard_rule,Sunday,57370b61a6855d70057417d1,20230910,20230910,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,f3a835c4-ce3a-4949-a27e-a2d6438ccd6a,20230904,Bangalore,20230910


In [68]:
df[df['pickup_cluster'].isin(['Harlur']) & df['recommendation_date'].isin(['20230822'])]

,pickup_cluster,drop_cluster,start_time,end_time,pickup_hex8,drop_hex8,eligible_strategy,week_period,service_detail_id,start_date,end_date,created_at,updated_at,period,captain_payout,type,updated_by,start_distance,end_distance,rule_id,run_date,city,recommendation_date
129,Harlur,ALL,0,2359,886189246dfffff,ALL,hot_cold_surge,Tuesday,57370b61a6855d70057417d1,20230822,20230822,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,f563d51a-9898-4213-b75c-8e4f3dfcc975,20230904,Bangalore,20230822
130,Harlur,ALL,0,2359,886189246dfffff,ALL,surge_dashboard_rule,Tuesday,57370b61a6855d70057417d1,20230822,20230822,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,b373e9e6-76b7-4e5d-8f89-f84d31d0b670,20230904,Bangalore,20230822
131,Harlur,ALL,0,2359,886189246dfffff,ALL,mismatch,Tuesday,57370b61a6855d70057417d1,20230822,20230822,1693804751,1693804751,,0,order_acceptance,manoj.ravirajan@rapido.bike,0,100,4dd4b882-bbaf-4452-b999-4f8dc019723f,20230904,Bangalore,20230822


In [69]:
df.rule_id.nunique()

2268

In [70]:
df.groupby(['city']).agg({'pickup_hex8':'nunique',})

,pickup_hex8
city,
Bangalore,36


In [71]:
df.groupby(['pickup_cluster','drop_cluster']).agg({'eligible_strategy':set, \
                                                   'pickup_hex8' : 'nunique', \
                                                   'start_date' : 'nunique'
                                                  })

,,eligible_strategy,pickup_hex8,start_date
pickup_cluster,drop_cluster,,,
BGS Kengeri,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Bagalur,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Banaswadi,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",2,21
Bommasandra Industrial Area,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Domlur,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Electronic City Phase 2,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
HBR Layout,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Harlur,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",1,21
Horamavu,ALL,"{surge_dashboard_rule, mismatch, hot_cold_surge}",4,21


In [72]:
df.groupby(['pickup_cluster','drop_cluster']).agg({'pickup_hex8': set, \
                                                   'drop_hex8' : set, \
                                                   #'start_date' : set, \
                                                    'start_date' : 'nunique'
                                                  })

,,pickup_hex8,drop_hex8,start_date
pickup_cluster,drop_cluster,,,
BGS Kengeri,ALL,{8860145ad1fffff},{ALL},21
Bagalur,ALL,{8861892db5fffff},{ALL},21
Banaswadi,ALL,"{8861892ee5fffff, 8861892e85fffff}",{ALL},21
Bommasandra Industrial Area,ALL,{8861892645fffff},{ALL},21
Domlur,ALL,{8861892ed9fffff},{ALL},21
Electronic City Phase 2,ALL,{8861892665fffff},{ALL},21
HBR Layout,ALL,{8861892eb7fffff},{ALL},21
Harlur,ALL,{886189246dfffff},{ALL},21
Horamavu,ALL,"{8861892f13fffff, 8861892c0bfffff, 8861892c11fffff, 8861892c55fffff}",{ALL},21


In [73]:
df.groupby(['pickup_cluster','drop_cluster']).agg({'pickup_hex8': set, \
                                                   'eligible_strategy':set, \
                                                   #'start_date' : set, \
                                                    'start_date' : 'nunique'
                                                  })

,,pickup_hex8,eligible_strategy,start_date
pickup_cluster,drop_cluster,,,
BGS Kengeri,ALL,{8860145ad1fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Bagalur,ALL,{8861892db5fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Banaswadi,ALL,"{8861892ee5fffff, 8861892e85fffff}","{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Bommasandra Industrial Area,ALL,{8861892645fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Domlur,ALL,{8861892ed9fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Electronic City Phase 2,ALL,{8861892665fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
HBR Layout,ALL,{8861892eb7fffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Harlur,ALL,{886189246dfffff},"{surge_dashboard_rule, mismatch, hot_cold_surge}",21
Horamavu,ALL,"{8861892f13fffff, 8861892c0bfffff, 8861892c11fffff, 8861892c55fffff}","{surge_dashboard_rule, mismatch, hot_cold_surge}",21


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268 entries, 0 to 2267
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   pickup_cluster       2268 non-null   object  
 1   drop_cluster         2268 non-null   object  
 2   start_time           2268 non-null   object  
 3   end_time             2268 non-null   object  
 4   pickup_hex8          2268 non-null   object  
 5   drop_hex8            2268 non-null   object  
 6   eligible_strategy    2268 non-null   object  
 7   week_period          2268 non-null   object  
 8   service_detail_id    2268 non-null   object  
 9   start_date           2268 non-null   object  
 10  end_date             2268 non-null   object  
 11  created_at           2268 non-null   int64   
 12  updated_at           2268 non-null   int64   
 13  period               2268 non-null   object  
 14  captain_payout       2268 non-null   int64   
 15  type                 

In [687]:
test_df = df.groupby(['pickup_cluster']).agg({'pickup_hex8': set, \
                                                   #'eligible_strategy':set, \
                                                   #'start_date' : set, \
                                                    'start_date' : set
                                                  })
test_df

,pickup_hex8,start_date
pickup_cluster,,
BGS Kengeri,{8860145ad1fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Bagalur,{8861892db5fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Banaswadi,"{8861892ee5fffff, 8861892e85fffff}","{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Bommasandra Industrial Area,{8861892645fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Domlur,{8861892ed9fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Electronic City Phase 2,{8861892665fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
HBR Layout,{8861892eb7fffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Harlur,{886189246dfffff},"{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."
Horamavu,"{8861892c11fffff, 8861892c0bfffff, 8861892c55fffff, 8861892f13fffff}","{20230827, 20230830, 20230826, 20230829, 20230822, 20230903, 20230828, 20230908, 20230831, 20230904, 20230907, 20230821, 20230902, 20230824, 20230906, 20230909, 20230901, 20230910, 20230825, 20230..."


In [688]:
test_df.to_csv('exp_test_hex_&_dates.csv', index = False)

In [689]:
df = pd.read_parquet('/Users/rapido/Documents/code-repository/PRICING/Affluence/Experiment/experiment_documents')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268 entries, 0 to 2267
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   pickup_cluster       2268 non-null   object  
 1   drop_cluster         2268 non-null   object  
 2   start_time           2268 non-null   object  
 3   end_time             2268 non-null   object  
 4   pickup_hex8          2268 non-null   object  
 5   drop_hex8            2268 non-null   object  
 6   eligible_strategy    2268 non-null   object  
 7   week_period          2268 non-null   object  
 8   service_detail_id    2268 non-null   object  
 9   start_date           2268 non-null   object  
 10  end_date             2268 non-null   object  
 11  created_at           2268 non-null   int64   
 12  updated_at           2268 non-null   int64   
 13  period               2268 non-null   object  
 14  captain_payout       2268 non-null   int64   
 15  type                 

In [75]:
df1 = pd.read_parquet('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/test_existing_file.parquet')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pickup_cluster     9 non-null      object
 1   drop_cluster       9 non-null      object
 2   start_time         9 non-null      object
 3   end_time           9 non-null      object
 4   pickup_hex8        9 non-null      object
 5   drop_hex8          9 non-null      object
 6   eligible_strategy  9 non-null      object
 7   week_period        9 non-null      object
 8   service_detail_id  9 non-null      object
 9   start_date         9 non-null      object
 10  end_date           9 non-null      object
 11  created_at         9 non-null      int64 
 12  updated_at         9 non-null      int64 
 13  period             9 non-null      object
 14  captain_payout     9 non-null      int64 
 15  type               9 non-null      object
 16  updated_by         9 non-null      object
 17  s

In [76]:
cluster_hex = df.groupby(['pickup_cluster', 'pickup_hex8']).agg({'drop_hex8' : set}).reset_index()
cluster_hex = cluster_hex.drop(['drop_hex8'], axis=1)

In [77]:
cluster_hex.count()

pickup_cluster    36
pickup_hex8       36
dtype: int64

In [78]:
cluster_hex.head()

,pickup_cluster,pickup_hex8
0,BGS Kengeri,8860145ad1fffff
1,Bagalur,8861892db5fffff
2,Banaswadi,8861892e85fffff
3,Banaswadi,8861892ee5fffff
4,Bommasandra Industrial Area,8861892645fffff


In [79]:
#cluster_hex.to_csv('/Users/rapido/local-datasets/affluence/experiment/experiment_documents/selected_cluster_hex.csv', index = False)